# ELO Scoring Function

In [1]:
import numpy as np
import seaborn as sns
sns.set()
import pandas as pd
from pandas import DataFrame as df
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from collections import Counter

### Helper Functions

In [2]:
# Given eloA, and eloB, return the probability that A wins
def expected_prob(eloA, eloB):
    return 1 - (1 / (1+10**((eloA-eloB)/400)))

# Given eloA, eloB, constant = k, result of A. Return updated eloA, and eloB values
def update_elo(eloA, eloB, k, res):
    pa = expected_prob(eloA, eloB)
    eloA += (res - pa)*k
    eloB += ((1-res) - (1 - pa))*k
    return eloA, eloB

def percentage(num):
    return round(num,4)*100

### Finding ELO on Datasets

In [3]:
# Example of ELO Calculation

eloA = 1200
eloB = 1000
pa = expected_prob(eloA, eloB)
print('Chance of A to win = ', percentage(pa), '%' )

res = 1 # A wins
eloA, eloB = update_elo(eloA, eloB, 100, res)
print('ELO of A = ', round(eloA, 0))
print('ELO of B = ', round(eloB, 0))

Chance of A to win =  75.97 %
ELO of A =  1224.0
ELO of B =  976.0


For the ELO List, to improve efficiency could add players and update ELO as you run through the matches. For now, it initializes a list of all the players and the code pulls from the list.

In [4]:
def init_elo_df(trn_data):
    elo_df = df({'Player': [], 'Group': [], 'ELO': []})
    start_elo = 1000
    trn = trn_data[0]
    for player1 in trn:
        elo_df.loc[len(elo_df.index)] = [player1, 1, start_elo]
    for player2 in trn[player1]:
        elo_df.loc[len(elo_df.index)] = [player2, 2, start_elo]
    return elo_df

# Read pkl data
file_name = "C:/Users/drago/OneDrive - UW/Ratliff Research/tournaments/smallsoccer-tournaments.pkl"
trn_data = pd.read_pickle(file_name) # trn_data is a list w/ length of 4, and dicts inside w/ length of 12
# 1st level is 4 tournaments, 2nd level is 12 p1's, 3rd level is 12 p2's, 4th level is match histories

# Making ELO List, have a starter df with every player in every group, update ELO in each run,
# have the history pull elo from this list
elo_df = init_elo_df(trn_data)

# Output history dataframe (all data df) + ELO dataframe
hist_df = df({'Tournament': [], 'Player 1': [], 'Player 2': [], 'Return 1': [], 'Return 2': []})

k = 100
trnnum = 0
for trn in trn_data:
    trnnum += 1
    for player1 in trn:
        wins = 0
        draws = 0
        games = 0
        eloA = elo_df.loc[(elo_df['Player'] == player1) & (elo_df['Group'] == 1), 'ELO'].iloc[0] # Get ELO A
        for player2 in trn[player1]:
            eloB = elo_df.loc[(elo_df['Player'] == player2) & (elo_df['Group'] == 2), 'ELO'].iloc[0] # Get ELO B
            history = trn[player1][player2]
            for match in history:
                res = 0
                ret1 = match[0]
                ret2 = match[1]
                games += 1
                if (ret1 > ret2):
                    wins += 1
                    res = 1
                elif (ret1 == ret2):
                    draws += 1
                    res = 0.5
                new_eloA, new_eloB = update_elo(eloA, eloB, k, res) 
                #elo_df.replace([player1, 1, eloA], value=new_eloA) # Update ELO A
                #elo_df.replace([player2, 2, eloA], value=new_eloB) # Update ELO B
                hist_df.loc[len(hist_df.index)] = [trnnum, player1, player2, ret1, ret2]


OSError: [Errno 22] Invalid argument

In [5]:
def init_elo_df(trn_data):
    elo_df = df({'Player': [], 'Group': [], 'ELO': []})
    start_elo = 1000
    for idx, trn in enumerate(trn_data):
        if idx % 1 == 0:
            for player1 in trn:
                elo_df.loc[len(elo_df.index)] = [player1, 1, start_elo]
            for player2 in trn[player1]:
                elo_df.loc[len(elo_df.index)] = [player2, 2, start_elo]
    return elo_df

# Read pkl data
file_name = "C:/Users/drago/OneDrive - UW/Ratliff Research/tournaments/smallsoccer-tournaments.pkl"
trn_data = pd.read_pickle(file_name)
elo_df = init_elo_df(trn_data)

pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows', None)
player = player1
group = 1
print('Player = ', player)
print('Group = ', group)
print(elo_df.loc[(elo_df['Player'] == player) & (elo_df['Group'] == group), 'ELO'].iloc[0])
print(elo_df)

OSError: [Errno 22] Invalid argument

In [6]:
# Making ELO List, have a starter df with every player in every group, update ELO in each run,
# have the history pull elo from this list